Используемые тексты: "Маленький принц" и "Преступление и наказание"

In [90]:
partsCount = 400
myTestSize = 0.2

textDir = "./text/"
little_guy_File = textDir + "lg.txt"
punishment_File = textDir + "p.txt"
with open(little_guy_File, encoding="utf8") as f:
    little_guy_text = f.read()
    
with open(punishment_File, encoding="utf8") as f:
    punishment_text = f.read()

In [91]:
little_guy_text[:500]

'\ufeffМаленький принц\nI\nКогда мне было шесть лет, в книге под названием «Правдивые истории», где рассказывалось про девственные леса, я увидел однажды удивительную картинку. На картинке огромная змея — удав — глотала хищного зверя. Вот как это было нарисовано:\nВ книге говорилось: «Удав заглатывает свою жертву целиком, не жуя. После этого он уже не может шевельнуться и спит полгода подряд, пока не переварит пищу».\nЯ много раздумывал о полной приключений жизни джунглей и тоже нарисовал цветным карандаш'

In [92]:
punishment_text[:500]

'\ufeff     В начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в С-м переулке, на улицу и медленно, как бы в нерешимости, отправился к К-ну мосту.1\n     Он благополучно избегнул встречи с своею хозяйкой на лестнице. Каморка его приходилась под самою кровлей высокого пятиэтажного дома и походила более на шкаф, чем на квартиру. Квартирная же хозяйка его, у которой он нанимал эту каморку с обедом и прислугой, помещалась одною лест'

# Токенизация

Выделим из текстов отдельные слова. Удалим ненужные "слова" (знаки препинания, лишние символы)

In [93]:
import nltk

nltk.download("punkt")
little_guy_tokens = nltk.word_tokenize(little_guy_text)
little_guy_tokens[:15]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Irina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['\ufeffМаленький',
 'принц',
 'I',
 'Когда',
 'мне',
 'было',
 'шесть',
 'лет',
 ',',
 'в',
 'книге',
 'под',
 'названием',
 '«',
 'Правдивые']

In [94]:
# второй текст
punishment_tokens = nltk.word_tokenize(punishment_text)

punishment_tokens[:15]

['\ufeff',
 'В',
 'начале',
 'июля',
 ',',
 'в',
 'чрезвычайно',
 'жаркое',
 'время',
 ',',
 'под',
 'вечер',
 ',',
 'один',
 'молодой']

Воспользуемся стеммером, чтобы привести слова к одной форме.

In [95]:
from snowballstemmer import RussianStemmer
stemmer = RussianStemmer()
stemmerResult1 = stemmer.stemWords(little_guy_text.lower().split())
stemmerResult2 = stemmer.stemWords(punishment_text.lower().split())

print(stemmerResult1[:10])
print('..........................')
print(stemmerResult2[:10])

['\ufeffмаленьк', 'принц', 'i', 'когд', 'мне', 'был', 'шест', 'лет,', 'в', 'книг']
..........................
['\ufeff', 'в', 'начал', 'июля,', 'в', 'чрезвычайн', 'жарк', 'время,', 'под', 'вечер,']


Как видим, слова получились "кривыми", простое обрезание окончания - не вариант. Воспользуемся лемматизатором.

In [96]:
import pymystem3
mystem = pymystem3.Mystem()
little_guy_lemm = mystem.lemmatize(" ".join(little_guy_tokens))
punishment_lemm = mystem.lemmatize(" ".join(punishment_tokens))
print(little_guy_lemm[:10])
print('.............')
print(punishment_lemm[:10])

['\ufeff', 'маленький', ' ', 'принц', ' ', 'I', ' ', 'когда', ' ', 'я']
.............
['\ufeff ', 'в', ' ', 'начало', ' ', 'июль', ' , ', 'в', ' ', 'чрезвычайно']


Посмотрим на морфологический анализ слов.

In [97]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
lemm_pymorph1 = [morph.parse(s)[0].normal_form for s in little_guy_tokens]
lemm_pymorph2 = [morph.parse(s)[0].normal_form for s in punishment_tokens]
print(lemm_pymorph1[:10])
print('................')
print(lemm_pymorph2[:10])


['\ufeffмаленький', 'принц', 'i', 'когда', 'я', 'быть', 'шесть', 'год', ',', 'в']
................
['\ufeff', 'в', 'начало', 'июль', ',', 'в', 'чрезвычайно', 'жаркое', 'время', ',']


In [98]:
# Удалим знаки, пустые слова, числа

bad_chars = ["«", "»",".",",","—",":","1","2","3","4","5","6","7","8","9","0","?","-","[","]", "стр", "(", ")",
            ";", "X", "!", "издание", "…", "’","&"]

def containsStr(a,b):
    for i in b:
        if i in a:
            return True
    return False

def deleteWords(wordList):
    resultList = [item for item in wordList if len(item) > 1 ]
    resultList = [item for item in resultList if not containsStr(item, bad_chars) ]
    return resultList



little_guy_lemm_clear = deleteWords(little_guy_lemm)

punishment_lemm_clear = deleteWords(punishment_lemm)

In [99]:
print(little_guy_lemm_clear[:10])

['маленький', 'принц', 'когда', 'быть', 'шесть', 'год', 'книга', 'под', 'название', 'правдивый']


In [100]:
print(punishment_lemm_clear[:10])

['\ufeff ', 'начало', 'июль', 'чрезвычайно', 'жаркий', 'время', 'под', 'вечер', 'один', 'молодой']


Чтобы снова собрать из этого, текст, просто сконкатенируем все получившиеся строки:

In [101]:
little_guy_text_clear = " ".join(little_guy_lemm_clear)
punishment_text_clear = " ".join(punishment_lemm_clear)

In [102]:
little_guy_text_clear[:1000]

'маленький принц когда быть шесть год книга под название правдивый история где рассказываться про девственный лес увидеть однажды удивительный картинка на картинка огромный змея удав глотать хищный зверь вот как это быть нарисовать книга говориться удав заглатывать свой жертва целиком не жевать после это он уже не мочь шевельнуться спать полгода подряд пока не переваривать пища много раздумывать полный приключение жизнь джунгли тоже нарисовать цветной карандаш свой первый картинка это быть мой рисунок вот что нарисовать показывать мой творение взрослый спрашивать не ли они разве шляпа возражать это быть совсем не шляпа это быть удав который проглатывать слон тогда нарисовать удав изнутри чтобы взрослый быть понятно они ведь всегда нужно весь объяснять это мой рисунок взрослый посоветовать не рисовать змей ни снаружи ни изнутри много интересоваться география история арифметика правописание вот как случаться что шесть год отказываться от блестящий карьера художник потерпеть неудача рисун

In [103]:
punishment_text_clear[:1000]

'\ufeff  начало июль чрезвычайно жаркий время под вечер один молодой человек выходить из свой каморка который нанимать от жилец переулок на улица медленно как бы нерешимость отправляться ну мост он благополучно избегать свой хозяйка на лестница каморка его приходиться под сам кровля высокий пятиэтажный дом походить более на шкаф чем на квартира квартирный же хозяйка он который он нанимать этот каморка обед прислуга помещаться один лестница ниже отдельный квартира каждый раз при выход на улица он непременно надо быть проходить мимо хозяйкин кухня почти всегда настежь отворять на лестница каждый раз молодой человек проходить мимо чувствовать болезненный трусливый ощущение который стыдиться от который морщиться он быть должный кругом хозяйка бояться она не то чтоб он быть так трусливый забивать совсем даже напротив но некоторый время он быть раздражительный напряженный состояние похожий на ипохондрия он до то углубляться себя уединяться от все что бояться даже всякий не только хозяйка он 

In [104]:
# функция разделения на части
def chunkify(lst,n):
    return [lst[i::n] for i in range(n)]

In [105]:
little_guy_parts = chunkify(little_guy_lemm_clear, partsCount)
punishment_parts = chunkify(punishment_lemm_clear, partsCount)

punishment_parts[1][:20]

['начало',
 'грустный',
 'считать',
 'не',
 'верхний',
 'он',
 'минута',
 'затягивать',
 'раскольников',
 'голова',
 'собственный',
 'назад',
 'обращать',
 'его',
 'вставать',
 'не',
 'тот',
 'он',
 'болезненный',
 'писать']

In [106]:
# объединяем слова в частях
# на выходе - список из текстов
def joinWordsToTextList(wordsLists):
    result = []
    for i in wordsLists:
        oneText = " ".join(i)
        result.append(oneText)
    return result


little_guy_texts = joinWordsToTextList(little_guy_parts)
punishment_texts = joinWordsToTextList(punishment_parts)

In [107]:
import os
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=["text","class"])

for i in little_guy_texts:
    df = df.append(pd.Series({"text":i, "class":0}), ignore_index=True)

# у второго типа текстов класс == 1
for i in punishment_texts:
    df = df.append(pd.Series({"text":i, "class":1}), ignore_index=True)
    

print("little_guy_texts size:"+str(len(little_guy_texts[3])), "punishment_text size:"+str(len(punishment_texts[3])))



little_guy_texts size:165 punishment_text size:1216


In [108]:
df

,text,class
0,маленький портрет это ли планета решаться же б...,0
1,принц какой самолет он то изменять ничто они о...,0
2,когда после мой бабочка как свой не раскрывать...,0
3,быть удаваться самолет они он обычный думать п...,0
4,шесть нарисовать он спрашивать она сдержанност...,0
...,...,...
795,несмотря безобразный что как он один бойко пут...,1
796,на '' вы она относить этот изредка для бы по...,1
797,будний мечта быть отпирать то воздух только чт...,1
798,время даже отчетливо комод что можно место же ...,1


In [109]:
# разбив на трейн и тест
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["text"], df["class"], test_size=myTestSize, random_state=42)

In [110]:
X_train

264    они отказываться разглядеть ничто клочок такой...
615    хорошо шляпник из из лестница похожий солидно ...
329    хватить барашек турецкий один путешествовать н...
342    тысяча ка докладывать цвет быть молчание цвето...
394    серьезно это какой стареть большой ты занимать...
                             ...                        
71     мой или тихо принц вышина неотложный планета п...
106    они ты полупризнание несмышленый бывать пойти ...
270    галстук барашек давать еще поздно который его ...
435    лестница даже выходить неприятный шкатулка ста...
102    чтобы речь мой на сперва сказать делать таинст...
Name: text, Length: 640, dtype: object

In [111]:
X_test

696    пожалуй непременно крошечный почитай такой на ...
667    трусость так только процент но он напиток это ...
63     нарисовать ничуть сюда же есть когда тряхнуть ...
533    он серединный действительно два сделать мещани...
66     свой быть но сказать баобаб вдохновлять его го...
                             ...                        
589    вздор то только любопытство она свой манишка о...
798    время даже отчетливо комод что можно место же ...
744    '  шаг на молодой раз смазывать его деньги же ...
513    чтоб быть этот состоять молодой ватага рост не...
670    аксиома он сверкать месяц мочь от всегда задав...
Name: text, Length: 160, dtype: object

# Bag-of-Words-эмбеддинг
на всех данных обучние

In [112]:
from sklearn.feature_extraction.text import CountVectorizer

bof_vect = CountVectorizer()
bof_vect.fit(np.hstack([X_train, X_test]))
bof_train = bof_vect.transform(X_train)
bof_test = bof_vect.transform(X_test)

In [113]:
# пространство признаков
bof_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [114]:
bof_train.toarray().shape

(640, 8329)

## TF-IDF-Эмбеддинг 

In [115]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(np.hstack([X_train, X_test]))
tfidf_train = tfidf_vect.transform(X_train)
tfidf_test = tfidf_vect.transform(X_test)

In [116]:
tfidf_train.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [117]:
tfidf_train.toarray().shape

(640, 8329)

# Word2vec-эмбеддинг

In [118]:
from gensim.models import Word2Vec

X_train_w2v = X_train.apply(str.split)
X_test_w2v = X_test.apply(str.split)
w2v_vect = Word2Vec(np.hstack([X_train_w2v, X_test_w2v]), size=160, min_count=10)

In [119]:
X_train_w2v

264    [они, отказываться, разглядеть, ничто, клочок,...
615    [хорошо, шляпник, из, из, лестница, похожий, с...
329    [хватить, барашек, турецкий, один, путешествов...
342    [тысяча, ка, докладывать, цвет, быть, молчание...
394    [серьезно, это, какой, стареть, большой, ты, з...
                             ...                        
71     [мой, или, тихо, принц, вышина, неотложный, пл...
106    [они, ты, полупризнание, несмышленый, бывать, ...
270    [галстук, барашек, давать, еще, поздно, которы...
435    [лестница, даже, выходить, неприятный, шкатулк...
102    [чтобы, речь, мой, на, сперва, сказать, делать...
Name: text, Length: 640, dtype: object

In [120]:
w2v_vect.most_similar(positive="принц")

<ipython-input-120-db20acbdd9c3>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  w2v_vect.most_similar(positive="принц")


[('маленький', 0.9999072551727295),
 ('цветок', 0.9998862743377686),
 ('планета', 0.9998816251754761),
 ('мой', 0.9998763203620911),
 ('сказать', 0.9998661279678345),
 ('звезда', 0.9998599290847778),
 ('спрашивать', 0.9998593926429749),
 ('когда', 0.9998427033424377),
 ('если', 0.9998371601104736),
 ('тысяча', 0.9998350143432617)]

In [121]:
w2v_vect.most_similar(negative=["принц"])

<ipython-input-121-55be5be6cde4>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  w2v_vect.most_similar(negative=["принц"])


[('вырываться', -0.9987562894821167),
 ('полезть', -0.9988167881965637),
 ('направо', -0.9988741278648376),
 ('привидение', -0.9988819360733032),
 ('неподвижный', -0.9988837242126465),
 ('шум', -0.9988928437232971),
 ('пожелать', -0.9988932609558105),
 ('великий', -0.9988971948623657),
 ('барышня', -0.9989217519760132),
 ('прибывать', -0.9989558458328247)]

In [122]:
# как представлено слово принц
w2v_vect["принц"]

<ipython-input-122-211e3f66d4ae>:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  w2v_vect["принц"]


array([ 0.245537  ,  0.03628403, -0.18329716, -0.19074942, -0.29679808,
        0.08081293, -0.12029052, -0.41935748, -0.22911836, -0.01181113,
       -0.20695761, -0.0256241 ,  0.13900298,  0.1437377 , -0.6162996 ,
       -0.11384013,  0.04442195,  0.03383857,  0.02422381,  0.54271865,
       -0.24837394, -0.45068517, -0.12207114,  0.28249276, -0.05764626,
       -0.1623938 ,  0.02498499, -0.3239144 , -0.0063332 ,  0.4242198 ,
        0.26691142,  0.13801765, -0.41811973, -0.18957224, -0.1652076 ,
        0.35551426, -0.16210091, -0.36003035, -0.12139721,  0.2020953 ,
       -0.16712724, -0.14203514, -0.25931618, -0.52077097,  0.15856254,
       -0.35105705, -0.36204523,  0.00360463,  0.10166933, -0.36262202,
        0.11752865, -0.07634296, -0.07751606, -0.10226088, -0.09196793,
        0.14762802,  0.30522782,  0.13183914, -0.05922295, -0.09179928,
        0.13197201, -0.12706925,  0.01360625, -0.06621646, -0.32570153,
        0.09541529, -0.15323287, -0.15080276,  0.09860469, -0.24

In [123]:
# усредним векторы слов
def text2vec(text):
    vecs = []
    for word in text:
        try:
            vecs.append(w2v_vect[word])
        except KeyError:
            pass
    return np.sum(vecs, axis=0) / len(vecs)

    
    
w2v_train = X_train_w2v.apply(text2vec)
w2v_test = X_test_w2v.apply(text2vec)

w2v_train

<ipython-input-123-a31dbd1028e5>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vecs.append(w2v_vect[word])


264    [0.192212, 0.025896449, -0.14253318, -0.146977...
615    [0.19883347, 0.025716836, -0.14745197, -0.1508...
329    [0.20977738, 0.02752598, -0.15580465, -0.15948...
342    [0.19421, 0.024559291, -0.14399609, -0.1479173...
394    [0.18944237, 0.0257835, -0.1401576, -0.1439752...
                             ...                        
71     [0.1983576, 0.02610941, -0.14667837, -0.150666...
106    [0.20391141, 0.026656128, -0.15144986, -0.1547...
270    [0.19439805, 0.025179038, -0.14384255, -0.1478...
435    [0.19564474, 0.025394067, -0.14503337, -0.1484...
102    [0.22170496, 0.029774982, -0.1645189, -0.16840...
Name: text, Length: 640, dtype: object

In [124]:
w2v_train = np.dstack(w2v_train)[0]

In [125]:
w2v_train

array([[ 0.192212  ,  0.19883347,  0.20977738, ...,  0.19439805,
         0.19564474,  0.22170496],
       [ 0.02589645,  0.02571684,  0.02752598, ...,  0.02517904,
         0.02539407,  0.02977498],
       [-0.14253318, -0.14745197, -0.15580465, ..., -0.14384255,
        -0.14503337, -0.1645189 ],
       ...,
       [ 0.31418163,  0.32424074,  0.34152094, ...,  0.3165012 ,
         0.31912684,  0.36048463],
       [ 0.20235014,  0.20952162,  0.22028418, ...,  0.2038176 ,
         0.20634198,  0.2322079 ],
       [-0.25342125, -0.26163012, -0.27671805, ..., -0.25575238,
        -0.25775585, -0.29120427]], dtype=float32)

# Классификация текстов
Для каждого эмбеддинга вычислим по два суммарных вектора - для текстов маленького принца и преступления и наказания.

### Для Bag-of-Words

In [126]:
# средний текст маленького принца
prinz_mean_bof = np.sum( bof_train[y_train == 0], axis=0 )
prinz_mean_bof.shape

(1, 8329)

In [127]:
# средний текст преступления и наказания
punishment_mean_bof = np.sum( bof_train[y_train == 1], axis=0 )
punishment_mean_bof.shape

(1, 8329)

### Для TF-IDF

In [128]:
prinz_mean_tfidf = np.sum(tfidf_train[y_train == 0], axis=0 )

punishment_mean_tfidf = np.sum(tfidf_train[y_train == 1], axis=0 )

prinz_mean_tfidf.shape

(1, 8329)

### Для Word2Vec

In [129]:
prinz_mean_w2v = np.sum(w2v_train[:,y_train==0], axis=1)

punishment_mean_w2v = np.sum(w2v_train[:,y_train==1], axis=1)

prinz_mean_w2v.shape

(160,)

In [130]:
prinz_mean_w2v

array([ 6.40624084e+01,  8.44387722e+00, -4.75578613e+01, -4.88064499e+01,
       -7.94660797e+01,  2.23308926e+01, -3.10764503e+01, -1.10098740e+02,
       -6.01521187e+01, -3.66172647e+00, -5.51797066e+01, -7.77440453e+00,
        3.68218803e+01,  3.81487160e+01, -1.60721985e+02, -2.95013008e+01,
        1.02565041e+01,  9.78651714e+00,  8.11316776e+00,  1.45240921e+02,
       -6.54358521e+01, -1.18274536e+02, -3.26296120e+01,  7.44717102e+01,
       -1.32945814e+01, -4.26390572e+01,  7.79782772e+00, -8.57362900e+01,
       -2.73989820e+00,  1.11015869e+02,  7.08633423e+01,  3.84529495e+01,
       -1.11289978e+02, -4.90402260e+01, -4.25191002e+01,  9.25432663e+01,
       -4.47349243e+01, -9.69982758e+01, -3.20307350e+01,  5.40704536e+01,
       -4.38650017e+01, -3.88428612e+01, -6.74242477e+01, -1.37171616e+02,
        4.27638474e+01, -9.12902908e+01, -9.72430954e+01,  1.03766370e+00,
        2.66416454e+01, -9.65082779e+01,  3.16157379e+01, -1.89150543e+01,
       -1.95861568e+01, -

Построим датафреймы с результатами классификаций тестовых текстов. Решение - по косинусному расстоянию.

In [131]:
from scipy.spatial.distance import cosine

bof_prinz = np.apply_along_axis(cosine, 1, bof_test.toarray(), v=prinz_mean_bof)
bof_punishment = np.apply_along_axis(cosine, 1, bof_test.toarray(), v=punishment_mean_bof)

bof_results = pd.DataFrame ([
    bof_prinz,
    bof_punishment,
    np.maximum(bof_prinz, bof_punishment) != bof_punishment,
    y_test
], index=["liitle_prinz", "punishment", "predict", "class"]).T.astype(np.float)

bof_results

,liitle_prinz,punishment,predict,class
0,0.412849,0.306967,1.0,1.0
1,0.431667,0.386648,1.0,1.0
2,0.616548,0.609311,1.0,0.0
3,0.381547,0.261236,1.0,1.0
4,0.557969,0.656190,0.0,0.0
...,...,...,...,...
155,0.342913,0.255972,1.0,1.0
156,0.415083,0.346108,1.0,1.0
157,0.496763,0.410008,1.0,1.0
158,0.377861,0.259563,1.0,1.0


In [132]:
from sklearn.metrics import accuracy_score

accuracy_score(bof_results["predict"], bof_results["class"])

0.9125

In [133]:
tfidf_prinz = np.apply_along_axis(cosine, 1, tfidf_test.toarray(), v=prinz_mean_tfidf)
tfidf_punishment = np.apply_along_axis(cosine, 1, tfidf_test.toarray(), v=punishment_mean_tfidf)

tfidf_results = pd.DataFrame ([
    tfidf_prinz,
    tfidf_punishment,
    np.maximum(tfidf_prinz, tfidf_punishment) != tfidf_punishment,
    y_test
], index=["little_prinz", "punishment", "predict", "class"]).T.astype (np.float)

tfidf_results

,little_prinz,punishment,predict,class
0,0.737594,0.619104,1.0,1.0
1,0.749908,0.681945,1.0,1.0
2,0.842430,0.858291,0.0,0.0
3,0.704611,0.557931,1.0,1.0
4,0.765273,0.853293,0.0,0.0
...,...,...,...,...
155,0.714353,0.603972,1.0,1.0
156,0.743852,0.635021,1.0,1.0
157,0.785575,0.680801,1.0,1.0
158,0.702591,0.576456,1.0,1.0


In [134]:
accuracy_score(tfidf_results["predict"], tfidf_results["class"])

0.96875

In [135]:
# w2v

w2v_prinz = np.apply_along_axis(cosine, 0, w2v_test, v=prinz_mean_w2v)
w2v_punishment = np.apply_along_axis(cosine, 0, w2v_test, v=punishment_mean_w2v)

w2v_results = pd.DataFrame ([
    tfidf_prinz,
    tfidf_punishment,
    np.maximum(w2v_prinz, w2v_punishment) != w2v_punishment,
    y_test
], index=["little_prinz", "punishment", "predict", "class"]).T.astype (np.float)

w2v_results


,little_prinz,punishment,predict,class
0,0.737594,0.619104,1.0,1.0
1,0.749908,0.681945,1.0,1.0
2,0.842430,0.858291,0.0,0.0
3,0.704611,0.557931,0.0,1.0
4,0.765273,0.853293,0.0,0.0
...,...,...,...,...
155,0.714353,0.603972,0.0,1.0
156,0.743852,0.635021,1.0,1.0
157,0.785575,0.680801,1.0,1.0
158,0.702591,0.576456,1.0,1.0


In [136]:
accuracy_score(w2v_results["predict"], w2v_results["class"])

0.50625

##### Попробуем применить классическую модель поверх эмбеддингов

In [137]:
from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier().fit(tfidf_train.toarray(), y_train.tolist()).score(tfidf_test.toarray(), y_test.tolist())

1.0

In [138]:
RandomForestClassifier().fit(bof_train.toarray(), y_train.tolist()).score(bof_test.toarray(), y_test.tolist())

1.0